In [4]:
#### Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time   # 시간 관련 모듈
import sys    # txt파일 저장관련 모듈
import math   # 페이지 번호 계산위해 사용 모듈
import pandas as pd   
import os
import urllib.request
import urllib
import re

# Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("아마존 Best Seller 상품 정보 추출하기 ")
print("=" *80)

query_txt = '아마존'
query_url='https://www.amazon.com/Best-Sellers-Beauty-Personal-Care/zgbs/beauty/ref=zg_bs_pg_1_beauty?_encoding=UTF8&pg=1&language=ko_KR'

cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 50)

f_dir = input('2.파일을 저장할 폴더명만 쓰세요(기본값:c:\\py_temp\\):')
if f_dir =='' :
    f_dir = "c:\\py_temp2\\"

# Step 3.저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, \
                                      now.tm_hour, now.tm_min, now.tm_sec)

img_dir = f_dir+s+'-'+query_txt+"\\images"

os.makedirs(img_dir)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xlsx'

#Step 4. 크롬 드라이버를 지정하고 웹사이트에 접속합니다
options = Options()
driver = webdriver.Chrome(options=options)


driver.get(query_url)
time.sleep(1)
driver.maximize_window()
time.sleep(3)




def scroll_down(driver):      
    driver.execute_script("window.scrollBy(0,1000);")
    time.sleep(4)
    

        
#Step 5.수집 내용을 저장하기 위해 빈 리스트를 지정합니다
ranking2=[] 
url2 = []     # 판매 순위 지정용 리스트
title2=[]      # 제품 이름 저장용 리스트
s_price2=[]    # 현재판매가격 저장용 리스트
star_point2=[]   # 할인율 저장용 리스트
review_cnt2 = []
content2 = []
content_2 = []
#Step 6. 웹 페이지에서 정보를 추출합니다


#Step 7-1.이미지 추출하기


img_file_no = 1

for x in range(1, page_cnt + 1) :
    for i in range(1, 6) :
        scroll_down(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    slist = soup.find('ol','a-ordered-list').find_all('li' , "zg-no-numbers")  
    
    #Step 7-2. 텍스트 추출하기
       # 판매순위용 변수
    count = 1    
    for li in slist:
        os.chdir(img_dir)
        if count > cnt :
            break            
            
    
        try :
              photo = li.find('span','a-list-item').find('img')['src']
        except AttributeError :
            continue
        urllib.request.urlretrieve(photo,str(img_file_no)+'.jpg')
        time.sleep(1)
    
        img_file_no += 1
        
                
        f = open(ff_name, 'a',encoding='UTF-8')
        f.write("-----------------------------------------------------"+"\n")
        print("-" *70)

        
        url_1 = li.find("a")["href"]
        url = "https://www.amazon.com" + url_1
        
        print('1.url:',url.replace("\n",""))
        f.write('1.url:'+ url + "\n")
        url2.append(url)
        
        #판매순위
        sid = 'no' + str(count)
        try :
            ranking = li.find('span','zg-bdg-text').get_text()
            ranking_1 = ranking.replace("#" , "")
            
        except AttributeError :
            ranking = ''
        
        print('1.판매순위:',ranking_1.replace("\n",""))
        f.write('1.판매순위:'+ ranking_1 + "\n")
        ranking2.append(ranking_1)
    
        #제품이름
        try :
            title = li.find('div','_cDEzb_p13n-sc-css-line-clamp-3_g3dy1').get_text()
        except :
            title = li.find('div','_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y').get_text()
    
        print("2.제품이름:", title.replace("\n",""))
        f.write('2.제품이름:'+ title + "\n")
        title2.append(title.replace("\n",""))
    
    
        
        #판매 가격
        try :
            s_price = li.find('span', '_cDEzb_p13n-sc-price_3mJ9Z').get_text()
        except :
            s_price = li.find('span','p13n-sc-price').get_text()
            
        print("3.판매가격:", s_price.replace("\n",""))
        f.write('3.판매가격:'+ s_price + "\n")
        s_price2.append(s_price.replace("\n",""))
    
        #할인율
        try :
            star_point = li.find('span', 'a-icon-alt').get_text()
            star_point1_1 = re.search("\d\.\d+" , star_point).group()
            
        except :
            star_point1_1 = '0.0'
    
        print("4.별점:", star_point1_1.replace("\n",""))
        f.write('4.별점:'+ star_point1_1 + "\n")
        star_point2.append(star_point1_1.replace("\n","")) 
    
        try :
            review_cnt = li.find("div" , "a-icon-row").find("span" , "a-size-small").get_text()
        except :
            review_cnt = "0"
        print("5.리뷰개수:" , review_cnt.replace("\n" , ""))
        f.write("5.리뷰개수:" + review_cnt + "\n")
        review_cnt2.append(review_cnt.replace("\n" , ""))

        href = li.find("a")["href"]
        driver.execute_script("window.open(arguments[0]);", href)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(3)
        
        html2 = driver.page_source
        soup2 = BeautifulSoup(html2, 'html.parser')
        time.sleep(3)
        

       
        try:

            content = soup2.find("ul" , "a-unordered-list a-vertical a-spacing-mini").get_text()
        
        except :
            content = ""
        

        print("6.상세내용:", content.replace("\n",""))
        f.write('6.상세내용:'+ content + "\n")
        content2.append(content.replace("\n",""))  

        try :
        
            content_1 = soup2.find("div" , "a-section a-spacing-small a-spacing-top-small").get_text()
        
        except :
            content_1 = ""
        

        print("7.상세내용2:", content_1.replace("\n",""))
        f.write('7.상세내용2:'+ content_1 + "\n")
        content_2.append(content_1.replace("\n",""))  
        
        driver.close()        
            
        driver.switch_to.window(driver.window_handles[0])     
                            
        count += 1
        
        if count > cnt :
            break
          
    
        time.sleep(0.5)

    x += 1       
    try :
        time.sleep(2)
        driver.find_element(By.LINK_TEXT, '%s' %x).click() # 다음 페이지번호 클릭
    except :
        break
     

# DataFrame을 컬럼 순서 지정해서 생성
g_best_seller = pd.DataFrame({
    '판매순위': ranking2,
    '이미지': [''] * len(ranking2),   # ★ 이미지 전용 빈 컬럼 (두 번째)
    '상품이름': title2,
    '판매가격': s_price2,
    '별점': star_point2,
    '리뷰개수': review_cnt2,
    '상세내용': content2,
    '상세내용2': content_2 , 
    'url' : url2 ,
})

# csv 저장
g_best_seller.to_csv(fc_name, encoding="utf-8-sig", index=False)

# 엑셀 저장
g_best_seller.to_excel(fx_name, index=False)


print("\n") 
print("=" *80)
print("2.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("3.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("4.파일 저장 완료: xlsx 파일명 : %s " %fx_name)
print("=" *80)
     
# Step 9. 이미지 삽입하기 (엑셀 프로그램 없이)
# pip install openpyxl pillow
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as XLImage

# 기존에 저장한 xlsx 열기
wb = load_workbook(fx_name)
ws = wb["Sheet1"]  # pandas의 기본 시트명

# 이미지 컬럼(B열) 설정
ws.column_dimensions['B'].width = 20

# 행 높이 조정
row_cnt = cnt + 1
for r in range(2, row_cnt + 1):
    ws.row_dimensions[r].height = 100

# 이미지 삽입 (B열)
for i in range(1, cnt + 1):
    img_path = os.path.join(img_dir, f"{i}.jpg")
    cell_addr = f"B{i+1}"   # ★ 이미지가 여기로 들어감 (B열)

    if not os.path.exists(img_path):
        continue

    img = XLImage(img_path)
    img.width = 120
    img.height = 100
    ws.add_image(img, cell_addr)

wb.save(fx_name)

# (선택) 크롬 드라이버 종료


아마존 Best Seller 상품 정보 추출하기 


1.크롤링 할 건수는 몇건입니까?:  100
2.파일을 저장할 폴더명만 쓰세요(기본값:c:\py_temp\): 


----------------------------------------------------------------------
1.url: https://www.amazon.com/Mighty-Patch-Hydrocolloid-Absorbing-count/dp/B074PVTPBW/ref=zg_bs_g_beauty_d_sccl_1/145-7716852-6267456?psc=1
1.판매순위: 1
2.제품이름: Mighty Patch™ Original patch from Hero Cosmetics – The #1 Hydrocolloid Acne Pimple Patch for Shrinking Zits and Whiteheads in 1 use; Nighttime Spot Stickers for Face and Skin (36 Count)
3.판매가격: $19.00
4.별점: 4.6
5.리뷰개수: 178,712
6.상세내용:   THE #1 ACNE PATCH* IN THE GAME: The reason people keep coming back to the Mighty Patch Original patches? Because they work when it counts. *Based on unit sales, Circana total US Multi-Outlet, latest 52 weeks ending 05/04/2025.    SHRINKS THE LOOK OF PIMPLES WITH JUST 1 MIGHTY PATCH: As soon as you notice a whitehead, hit with a Mighty Patch and let it do its thing. After 6 hours, you can peel it right off, and voilà.    HELPS PROTECT PIMPLES FROM PICKING AND POPPING: Mighty Patch Original features medical grade hydrocolloid, whi

In [ ]:
print(content)